<a href="https://colab.research.google.com/github/yiailake/colab/blob/main/Z_Image_Turbo_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/comfyanonymous/ComfyUI

%cd /content/ComfyUI
!pip install -r requirements.txt

!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d /content/ComfyUI/models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d /content/ComfyUI/models/clip -o qwen_3_4b.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d /content/ComfyUI/models/vae -o ae.safetensors

Cloning into 'ComfyUI'...
remote: Enumerating objects: 28128, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 28128 (delta 8), reused 3 (delta 3), pack-reused 28107 (from 3)
Receiving objects: 100% (28128/28128), 74.76 MiB | 19.98 MiB/s, done.
Resolving deltas: 100% (19051/19051), done.
/content/ComfyUI
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 130.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 143.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 129.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.7/31.7 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 20.8 MB/s eta 0:00:0

In [4]:
%cd /content/ComfyUI

import os, random, time

import torch
import numpy as np
from PIL import Image

from nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    vae = VAELoader.load_vae("ae.safetensors")[0]

@torch.inference_mode()
def generate(input):
    tmp_dir = "/content/ComfyUI/output"
    os.makedirs(tmp_dir, exist_ok=True)

    values = input["input"]

    positive_prompt = values['positive_prompt']
    negative_prompt = values['negative_prompt']
    seed = values['seed']  # 0
    steps = values['steps']  # 9
    cfg = values['cfg']  # 1.0
    sampler_name = values['sampler_name']  # euler
    scheduler = values['scheduler']  # simple
    denoise = values['denoise']  # 1.0
    width = values['width']  # 1024
    height = values['height']  # 1024
    batch_size = values['batch_size']  # 每批生成的图片数量
    num_batches = values.get('num_batches', 1)  # 批次数，默认为1

    # 存储所有生成图片的路径
    results = []

    for batch_idx in range(num_batches):
        # 如果seed为0，则为每批生成随机种子
        current_seed = seed
        if seed == 0:
            random.seed(int(time.time()) + batch_idx)  # 添加batch_idx确保不同批次有不同的种子
            current_seed = random.randint(0, 18446744073709551615)

        # 编码提示词
        positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
        negative = CLIPTextEncode.encode(clip, negative_prompt)[0]

        # 生成潜空间图像
        latent_image = EmptyLatentImage.generate(width, height, batch_size=batch_size)[0]

        # 采样
        samples = KSampler.sample(unet, current_seed, steps, cfg, sampler_name, scheduler,
                                positive, negative, latent_image, denoise=denoise)[0]

        # 解码
        decoded = VAEDecode.decode(vae, samples)[0].detach()

        # 保存每一张图片
        decoded_images = np.array(decoded * 255, dtype=np.uint8)
        for img_idx in range(decoded_images.shape[0]):
            image_array = decoded_images[img_idx]
            image_filename = f"z_image_turbo_batch{batch_idx}_img{img_idx}.png"
            image_path = os.path.join(tmp_dir, image_filename)
            Image.fromarray(image_array).save(image_path)
            results.append(image_path)

    return results


/content/ComfyUI


In [ ]:
input_data = {
    "input": {
        "positive_prompt": "A young Chinese woman poses seductively in a dimly lit bedroom, in a moody photographic style.",
        "negative_prompt": "",
        "seed": 0,
        "steps": 9,
        "cfg": 1.0,
        "sampler_name": "euler",
        "scheduler": "simple",
        "denoise": 1.0,
        "width": 1024,
        "height": 1024,
        "batch_size": 2,      # 每批生成2张图片
        "num_batches": 3      # 生成3批，总共6张图片
    }
}

image_paths = generate(input_data)
print(f"Generated {len(image_paths)} images:")
for path in image_paths:
    print(path)


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]